# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 5 2022 9:00AM,243528,22,623214,"NBTY Global, Inc.",Executing
1,Aug 5 2022 8:53AM,243527,20,8261359,"ACI Healthcare USA, Inc.",Released
2,Aug 5 2022 8:44AM,243525,10,PRF-34689,Bio-PRF,Released
3,Aug 5 2022 8:44AM,243524,10,PRF-32877-BO1,Bio-PRF,Executing
4,Aug 5 2022 8:44AM,243524,10,PRF-34621,Bio-PRF,Released
5,Aug 5 2022 8:44AM,243524,10,PRF-34624,Bio-PRF,Executing
6,Aug 5 2022 8:44AM,243524,10,PRF-34628,Bio-PRF,Released
7,Aug 5 2022 8:44AM,243524,10,PRF-34632,Bio-PRF,Released
8,Aug 5 2022 8:44AM,243524,10,PRF-34638,Bio-PRF,Released
9,Aug 5 2022 8:44AM,243524,10,PRF-34641,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,243524,Released,8
30,243525,Released,1
31,243526,Released,1
32,243527,Released,1
33,243528,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243524,2.0,8.0
243525,NaN,1.0
243526,NaN,1.0
243527,NaN,1.0
243528,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243451,1.0,0.0
243455,0.0,1.0
243461,14.0,5.0
243464,1.0,12.0
243478,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243451,1,0
243455,0,1
243461,14,5
243464,1,12
243478,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243451,1,0
1,243455,0,1
2,243461,14,5
3,243464,1,12
4,243478,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243451,1,
1,243455,,1
2,243461,14,5
3,243464,1,12
4,243478,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 5 2022 9:00AM,243528,22,"NBTY Global, Inc."
1,Aug 5 2022 8:53AM,243527,20,"ACI Healthcare USA, Inc."
2,Aug 5 2022 8:44AM,243525,10,Bio-PRF
3,Aug 5 2022 8:44AM,243524,10,Bio-PRF
13,Aug 5 2022 8:42AM,243523,20,"ACI Healthcare USA, Inc."
14,Aug 5 2022 8:41AM,243526,10,MedStone Pharma LLC
15,Aug 5 2022 8:40AM,243521,10,ISDIN Corporation
39,Aug 5 2022 8:39AM,243522,10,ISDIN Corporation
52,Aug 5 2022 8:38AM,243520,10,ISDIN Corporation
71,Aug 5 2022 8:38AM,243519,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 5 2022 9:00AM,243528,22,"NBTY Global, Inc.",1,
1,Aug 5 2022 8:53AM,243527,20,"ACI Healthcare USA, Inc.",,1
2,Aug 5 2022 8:44AM,243525,10,Bio-PRF,,1
3,Aug 5 2022 8:44AM,243524,10,Bio-PRF,2,8
4,Aug 5 2022 8:42AM,243523,20,"ACI Healthcare USA, Inc.",,1
5,Aug 5 2022 8:41AM,243526,10,MedStone Pharma LLC,,1
6,Aug 5 2022 8:40AM,243521,10,ISDIN Corporation,,24
7,Aug 5 2022 8:39AM,243522,10,ISDIN Corporation,,13
8,Aug 5 2022 8:38AM,243520,10,ISDIN Corporation,,19
9,Aug 5 2022 8:38AM,243519,10,ISDIN Corporation,,55


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 5 2022 9:00AM,243528,22,"NBTY Global, Inc.",,1
1,Aug 5 2022 8:53AM,243527,20,"ACI Healthcare USA, Inc.",1,
2,Aug 5 2022 8:44AM,243525,10,Bio-PRF,1,
3,Aug 5 2022 8:44AM,243524,10,Bio-PRF,8,2
4,Aug 5 2022 8:42AM,243523,20,"ACI Healthcare USA, Inc.",1,
5,Aug 5 2022 8:41AM,243526,10,MedStone Pharma LLC,1,
6,Aug 5 2022 8:40AM,243521,10,ISDIN Corporation,24,
7,Aug 5 2022 8:39AM,243522,10,ISDIN Corporation,13,
8,Aug 5 2022 8:38AM,243520,10,ISDIN Corporation,19,
9,Aug 5 2022 8:38AM,243519,10,ISDIN Corporation,55,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 5 2022 9:00AM,243528,22,"NBTY Global, Inc.",,1
1,Aug 5 2022 8:53AM,243527,20,"ACI Healthcare USA, Inc.",1,
2,Aug 5 2022 8:44AM,243525,10,Bio-PRF,1,
3,Aug 5 2022 8:44AM,243524,10,Bio-PRF,8,2
4,Aug 5 2022 8:42AM,243523,20,"ACI Healthcare USA, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 5 2022 9:00AM,243528,22,"NBTY Global, Inc.",NaN,1.0
1,Aug 5 2022 8:53AM,243527,20,"ACI Healthcare USA, Inc.",1.0,NaN
2,Aug 5 2022 8:44AM,243525,10,Bio-PRF,1.0,NaN
3,Aug 5 2022 8:44AM,243524,10,Bio-PRF,8.0,2.0
4,Aug 5 2022 8:42AM,243523,20,"ACI Healthcare USA, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 5 2022 9:00AM,243528,22,"NBTY Global, Inc.",0.0,1.0
1,Aug 5 2022 8:53AM,243527,20,"ACI Healthcare USA, Inc.",1.0,0.0
2,Aug 5 2022 8:44AM,243525,10,Bio-PRF,1.0,0.0
3,Aug 5 2022 8:44AM,243524,10,Bio-PRF,8.0,2.0
4,Aug 5 2022 8:42AM,243523,20,"ACI Healthcare USA, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3409060,188.0,3.0
12,243497,0.0,1.0
15,243509,1.0,0.0
16,487025,2.0,0.0
19,730481,3.0,3.0
20,1217480,10.0,20.0
21,243483,1.0,0.0
22,243528,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3409060,188.0,3.0
1,12,243497,0.0,1.0
2,15,243509,1.0,0.0
3,16,487025,2.0,0.0
4,19,730481,3.0,3.0
5,20,1217480,10.0,20.0
6,21,243483,1.0,0.0
7,22,243528,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,188.0,3.0
1,12,0.0,1.0
2,15,1.0,0.0
3,16,2.0,0.0
4,19,3.0,3.0
5,20,10.0,20.0
6,21,1.0,0.0
7,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,188.0
1,12,Released,0.0
2,15,Released,1.0
3,16,Released,2.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Executing,3.0,1.0,0.0,0.0,3.0,20.0,0.0,1.0
Released,188.0,0.0,1.0,2.0,3.0,10.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Executing,3.0,1.0,0.0,0.0,3.0,20.0,0.0,1.0
1,Released,188.0,0.0,1.0,2.0,3.0,10.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Executing,3.0,1.0,0.0,0.0,3.0,20.0,0.0,1.0
1,Released,188.0,0.0,1.0,2.0,3.0,10.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()